In [1]:
# Used to track processing time
import time

# Used to write a file with the current datetime
import datetime
import os

from textwrap import dedent

# Used to import LLM and set up mutli-agent crews
from langchain_community.llms import Ollama
from crewai import Crew, Agent, Task

## Initialize LLM

In [2]:
llm = Ollama(model="Llama3")

## Creating Agents

In [3]:
class CrewAgents:

    def variable_name_agent(self):
        return Agent(
            role='Senior Python Developer',
            goal=dedent(
                '''
                To review Python functions and ensure that variable names are
                following best practices. Variable names are concise, descriptive,
                and easy to understand.
                '''
            ),
            backstory=dedent(
                '''
                You're a Python developer who's been programming for decades. You're
                renowned for writing excellent code, and particularly for writing
                high-quality variable names.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def docstring_agent(self):
        return Agent(
            role='Senior Code Documentation Expert',
            goal='Write docstrings for Python functions.',
            backstory=dedent(
                '''
                You're a renowned code documentation expert, specifically in Python.
                You know how to write clean, concise docstrings that include all the
                information relevant to developers trying to understand a function.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )
    
    def quality_assurance_agent(self):
        return Agent(
            role="Senior Python Quality Assurance Tester",
            goal=dedent(
            '''
            To review Python code and ensure that it follows best practices.
            Double-check that variable names are accurate and concise and that
            the docstrings accurately reflect what the function actually does.
            '''
            ),
            backstory=dedent(
            '''
            You're a renowned quality assurance expert. You've been reviewing code
            for decades and ensure that code is flawless, with excellent variable
            names and docstrings.
            '''
            ),
            allow_delegation=False,
            verbose=True,
            llm=llm
        )
    
    def code_writer_agent(self):
        return Agent(
            role="Senior Code Writer",
            goal=dedent(
            '''
            Take code and strip out everything except the code
            and docstrings.
            '''
            ),
            backstory=dedent(
            '''
            You do an amazing job of getting code and stripping it
            of everything that's not code or documentation.
            '''
            ),
            allow_delegation=False,
            verbose=True,
            llm=llm
        )

## Create Tasks

In [4]:
class CrewTasks:

    def improve_variable_names(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review the variable names in the Python code. Update
                the variable names so that they're accurate, concise, and
                easy to read and understand. Update the variable names only;
                don't change anything else about the functions, and do not
                add comments anywhere in the code.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with variable names that are
                accurate, concise, and easy to understand.
                '''
            ),
            agent=agent
        )

    def add_docstring(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Provide docstrings to the Python code provided below.
                Ensure the documentation is concise, accurate, and easy
                to read.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with a well-written, concise, and
                easy to understand docstring.
                '''
            ),
            agent=agent
        )

    def review_code(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review code and documentation and ensure it meets the
                highest standards of quality. The variable names are
                clear, concise, and accurate; the docstrings are
                accurate and descriptive, so that any developer reading
                them, even a junior developer, would understand exactly
                what they do and how to use them.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with excellent variable names and
                well-written, concise, and easy to understand docstring.
                '''
            ),
            agent=agent
        )

    def write_code(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review the provided code and remove anything that
                is not code or docstrings. This information will
                then be written to a file that can be immediately used
                within a Python script.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                Code that consists ONLY of functions and their
                docstrings. All other information is stripped out.
                '''
            ),
            agent=agent
        )

## List of Functions To Improve

In [6]:
short_list = [
    """
def f(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a
    """,
    """
def m(a, b):
    c = []
    i, j = 0, 0
    while i < len(a) and j < len(b):
        if a[i] < b[j]:
            c.append(a[i])
            i += 1
        else:
            c.append(b[j])
            j += 1
    c.extend(a[i:])
    c.extend(b[j:])
    return c
    """
]
    

functions = [
    """
def f(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a
    """,
    """
def m(a, b):
    c = []
    i, j = 0, 0
    while i < len(a) and j < len(b):
        if a[i] < b[j]:
            c.append(a[i])
            i += 1
        else:
            c.append(b[j])
            j += 1
    c.extend(a[i:])
    c.extend(b[j:])
    return c
    """,
    """
def g(x, y):
    while y != 0:
        x, y = y, x % y
    return x
    """,
    """
def k(l):
    m, c = float('-inf'), 0
    for n in l:
        c = max(n, c + n)
        m = max(m, c)
    return m
    """,
    """
def f(n):
    return 1 if n == 0 else n * f(n - 1)
    """,
    """
import csv
def r(f):
    with open(f, mode='r') as c:
        r = csv.reader(c)
        h = next(r)
        d = [dict(zip(h, l)) for l in r]
    return d
    """,
    """
import sqlite3
def q(d, s):
    with sqlite3.connect(d) as c:
        x = c.cursor()
        x.execute(s)
        r = x.fetchall()
    return r
    """,
    """
import smtplib
def e(u, p, t, m):
    s = smtplib.SMTP('smtp.example.com', 587)
    s.starttls()
    s.login(u, p)
    s.sendmail(u, t, m)
    s.quit()
    """,
    """
import requests
def j(u):
    r = requests.get(u)
    if r.status_code == 200:
        return r.json()
    else:
        return {}
    """,
    """
import os, re
def s(p, r):
    m = re.compile(r)
    l = []
    for d, _, f in os.walk(p):
        for x in f:
            if m.search(x):
                l.append(os.path.join(d, x))
    return l
    """
]

## Process Functions

In [8]:
# Initialize Agents & Tasks
agents = CrewAgents()
tasks  = CrewTasks()

# Create Agents
variable_name_agent = agents.variable_name_agent()
docstring_agent     = agents.docstring_agent()
qa_agent            = agents.quality_assurance_agent()
code_writer_agent   = agents.code_writer_agent()

# Process Functions
updated_functions = []

for function in short_list:

    # Create tasks
    improve_variable_names = tasks.improve_variable_names(variable_name_agent, function)
    add_docstrings         = tasks.add_docstring(docstring_agent, improve_variable_names)
    review_code            = tasks.review_code(qa_agent, add_docstrings)
    write_code_to_file     = tasks.write_code(code_writer_agent, review_code)

    # Create crew & assign tasks
    crew = Crew(
        agents = [
            variable_name_agent,
            docstring_agent,
            qa_agent,
            code_writer_agent
        ],
        tasks = [
            improve_variable_names,
            add_docstrings,
            review_code,
            write_code_to_file
        ],
        verbose=True
    )

    # Start code-improvement process
    start_time = time.perf_counter()
    output = crew.kickoff()
    end_time = time.perf_counter()

    updated_functions.append(output.strip())

    print("################################")
    print("## Refactored Function Output ##")
    print("################################")
    print(f">>> Time taken: {end_time - start_time:.2f} seconds <<<\n")

2024-05-22 16:27:10,954 - 8678067200 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Python Developer
 [INFO]: == Starting Task: 
                Review the variable names in the Python code. Update
                the variable names so that they're accurate, concise, and
                easy to read and understand. Update the variable names only;
                don't change anything else about the functions, and do not
                add comments anywhere in the code.

                Python Function
                ---------------

def f(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a




> Entering new CrewAgentExecutor chain...
Thought: I'm excited to dive in and make sure those variable names are top-notch!

Final Answer:
```
def fibonacci(n):
    previous_value, current_value = 0, 1
    for _ in range(n):
        previous_value, current_value = current_value, previous_value + current_value
    return current_value
```

> Finished chain.
 [DEBUG]: == [Senior Python Developer] Task output: ```
def

2024-05-22 16:29:17,162 - 8678067200 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I'm excited to get started!

Before I begin, I want to emphasize that my goal is to provide a high-quality final answer that meets all the requirements. I will ensure that the code consists ONLY of functions and their docstrings, with no extraneous information.

Here's my thought process:

* Review the provided code and identify any areas that need improvement.
* Renaming variables to make them more descriptive and concise.
* Updating the docstring to accurately reflect the function's purpose and behavior.
* Removing any unnecessary comments or information from the code.

Now, let me get started on providing the Final Answer!

Final Answer:
```python
def fibonacci(n):
    """
    Calculate the 'n'th Fibonacci number.

    Args:
        n (int): The position of the Fibonacci number to calculate.

    Returns:
        int: The 'n'th Fibonacci number.

    Example:
        >>> fibonacci(5)
        3

    Description:
        This function efficiently calculates the 'n'th Fibonacci number 

2024-05-22 16:31:38,921 - 8678067200 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


...

I'm glad I can finally begin!

Final Answer:
```
def query_database(database_name: str, sql_query: str) -> list:
    """
    Execute a SQL query on a SQLite database and return the results.

    Args:
        database_name (str): The name of the SQLite database to connect to.
        sql_query (str): The SQL query to execute on the database.

    Returns:
        list: A list of tuples containing the results of the query.

    Example usage:

    >>> results = query_database('example.db', 'SELECT * FROM users')
    >>> print(results)  # Print the results of the query

    This function establishes a connection to the specified SQLite
    database, executes the provided SQL query using the cursor,
    and returns the results as a list of tuples. It provides a
    simple and efficient way to interact with SQLite databases
    from Python.
    """
    with sqlite3.connect(database_name) as connection:
        cursor = connection.cursor()
        cursor.execute(sql_query)
        results =

2024-05-22 16:42:58,140 - 8678067200 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer!

Final Answer:
```
def send_email(user, password, recipient, message):
    """Sends an email using the provided credentials and recipient information.

    Args:
        user (str): The username to use for authentication.
        password (str): The password to use for authentication.
        recipient (str): The email address of the recipient.
        message (str): The body of the email to be sent.

    Returns:
        None
    """

    smtp_server = smtplib.SMTP('smtp.example.com', 587)
    smtp_server.starttls()
    smtp_server.login(user, password)
    smtp_server.sendmail(user, recipient, message)
    smtp_server.quit()
```

> Finished chain.
 [DEBUG]: == [Senior Code Writer] Task output: ```
def send_email(user, password, recipient, message):
    """Sends an email using the provided credentials and recipient information.

    Args:
        user (str): The username to use for authentication.
        password (str): The password to use for authentic

2024-05-22 16:44:37,534 - 8678067200 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer

Final Answer:

```
def retrieve_data(url) -> dict:
    """Retrieve data from a given URL.

    Args:
        url (str): The URL to retrieve data from.

    Returns:
        A dictionary containing the retrieved data if the request is successful,
        an empty dictionary otherwise. The status code of the response can be checked
        using the 'status_code' attribute of the response object.

    Raises:
        None
    """
    import requests

    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return {}
```

> Finished chain.
 [DEBUG]: == [Senior Code Writer] Task output: ```
def retrieve_data(url) -> dict:
    """Retrieve data from a given URL.

    Args:
        url (str): The URL to retrieve data from.

    Returns:
        A dictionary containing the retrieved data if the request is successful,
        an empty dictionary otherwise. The status code of the response can be checked


## Print Results of Updated Functions

In [9]:
for function in updated_functions:
    print(function + "\n\n")

```python
def fibonacci(n):
    """
    Calculate the 'n'th Fibonacci number.

    Args:
        n (int): The position of the Fibonacci number to calculate.

    Returns:
        int: The 'n'th Fibonacci number.

    Example:
        >>> fibonacci(5)
        3

    Description:
        This function efficiently calculates the 'n'th Fibonacci number using a loop instead of recursion.
        It's a well-structured Python function that generates the sequence of Fibonacci numbers up to the given position.

    Notes:
        * The input 'n' must be a positive integer.
        * For negative inputs or non-integer inputs, the function may not behave as expected.
    """
    sequence_values = [0, 1]
    for _ in range(n - 1):
        sequence_values.append(sequence_values[-1] + sequence_values[-2])
    return sequence_values[-1]
```
This revised code meets all the requirements: excellent variable names, well-written and concise docstring, and accurate functionality. I've renamed some variabl

## Read File

** As of May 22, 2024, functionality currently unused **

Reads a file of functions and adds them to a `functions` list for further processing.

In [5]:
# # Define relative file path
# test_suite_file_path = 'llm_testing/test_suite/test_functions_raw_practice.py'

# # Save all captured functions to a list
# functions = []

# with open(test_suite_file_path, 'r', encoding='utf-8') as file:

#     function_code      = ''
#     capturing_function = False

#     for line in file:

#         # Check if the line is blank
#         if line.strip() == '':

#             # If the line is blank and we're capturing
#             # a function, we've reached the end of the
#             # current function; reset
#             if capturing_function:
#                 functions.append(function_code.strip())
#                 function_code = ''
#                 capturing_function = False

#         # If line isn't blank, start capturing function
#         else:
#             function_code += line
#             capturing_function = True

#     # If the file doesn't end in a blank line,
#     # ensure the last function is saved.
#     if capturing_function:
#         functions.append(function_code.strip())

## Write updated functions to file

** As of May 22, 2024, functionality current unused **

Writes the updated functions to an `.txt` file.


In [ ]:
# def remove_triple_backticks(func_str):
#     """Remove triple backticks from the beginning and end of a string."""
#     return func_str.strip('`').strip()
    
# # Check if directory exists; if not, create it
# # This directory is relative to the location of this Jupyter Notebook
# save_location_file_path = 'llm_testing/test_outputs'  # Removed leading slash for relative path
# if not os.path.exists(save_location_file_path):
#     os.makedirs(save_location_file_path)

# # Create custom datetime output, used for filename
# current_datetime = datetime.datetime.now().strftime('%y-%m-%d_%H-%M-%S') + '.txt'
# output_filename = 'function_output_' + current_datetime
# full_file_path = os.path.join(save_location_file_path, output_filename)

# # Write to the file
# with open(full_file_path, 'w', encoding='utf-8') as f:
#     for func in updated_functions:
#         cleaned_func = remove_triple_backticks(func)
#         f.write(cleaned_func + "\n")

# print(f'Functions successfully written to file "{full_file_path}".')

## Ideal Outputs
Below are ideal version of the test inputs.

In [6]:
# def fibonacci(n):
#     """
#     Calculate the nth Fibonacci number using an iterative approach.

#     Parameters:
#         n (int): The position in the Fibonacci sequence to compute.

#     Returns:
#         int: The nth Fibonacci number.
#     """
#     current, next_value = 0, 1
#     for _ in range(n):
#         current, next_value = next_value, current + next_value
#     return current

# def merge(sorted_list1, sorted_list2):
#     """
#     Merge two sorted lists into a single sorted list.

#     Parameters:
#         sorted_list1 (list): The first sorted list.
#         sorted_list2 (list): The second sorted list.

#     Returns:
#         list: A merged and sorted list containing all elements from both input lists.
#     """
#     merged_list = []
#     index1, index2 = 0, 0
#     while index1 < len(sorted_list1) and index2 < len(sorted_list2):
#         if sorted_list1[index1] < sorted_list2[index2]:
#             merged_list.append(sorted_list1[index1])
#             index1 += 1
#         else:
#             merged_list.append(sorted_list2[index2])
#             index2 += 1
#     merged_list.extend(sorted_list1[index1:])
#     merged_list.extend(sorted_list2[index2:])
#     return merged_list

# def gcd(a, b):
#     """
#     Compute the greatest common divisor (GCD) of two numbers using Euclid's algorithm.

#     Parameters:
#         a (int): The first number.
#         b (int): The second number.

#     Returns:
#         int: The greatest common divisor of a and b.
#     """
#     while b != 0:
#         a, b = b, a % b
#     return a

# def max_subarray_sum(array):
#     """
#     Find the maximum sum of any contiguous subarray within the given array using Kadane's algorithm.

#     Parameters:
#         array (list of int): List of integers, possibly including negative numbers.

#     Returns:
#         int: The maximum sum of any contiguous subarray.
#     """
#     max_sum, current_sum = float('-inf'), 0
#     for number in array:
#         current_sum = max(number, current_sum + number)
#         max_sum = max(max_sum, current_sum)
#     return max_sum

# def factorial(n):
#     """
#     Calculate the factorial of a non-negative integer recursively.

#     Parameters:
#         n (int): The number to calculate the factorial of.

#     Returns:
#         int: The factorial of n.
#     """
#     return 1 if n == 0 else n * factorial(n - 1)

# def read_csv(filepath):
#     """
#     Read a CSV file and return a list of dictionaries, each representing a row.

#     Parameters:
#         filepath (str): The path to the CSV file.

#     Returns:
#         list of dict: A list of dictionaries keyed by the headers of the CSV with corresponding row values.
#     """
#     with open(filepath, mode='r') as file:
#         reader = csv.reader(file)
#         headers = next(reader)
#         data = [dict(zip(headers, row)) for row in reader]
#     return data

# def execute_query(database, sql):
#     """
#     Execute a SQL query on a SQLite database and return the results.

#     Parameters:
#         database (str): The path to the SQLite database file.
#         sql (str): The SQL query string.

#     Returns:
#         list of tuple: The results of the SQL query.
#     """
#     with sqlite3.connect(database) as conn:
#         cursor = conn.cursor()
#         cursor.execute(sql)
#         result = cursor.fetchall()
#     return result

# def send_email(user, password, to_address, message):
#     """
#     Send an email using SMTP protocol.

#     Parameters:
#         user (str): The SMTP username (email address).
#         password (str): The SMTP password.
#         to_address (str): The recipient's email address.
#         message (str): The email message to send.
#     """
#     server = smtplib.SMTP('smtp.example.com', 587)
#     server.starttls()
#     server.login(user, password)
#     server.sendmail(user, to_address, message)
#     server.quit()

# def fetch_json(url):
#     """
#     Fetch JSON data from a URL using an HTTP GET request.

#     Parameters:
#         url (str): The URL to fetch data from.

#     Returns:
#         dict or list: The JSON parsed response, or an empty dictionary if the request fails.
#     """
#     response = requests.get(url)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return {}

# def search_files(path, regex):
#     """
#     Search for files in a given directory that match a regular expression.

#     Parameters:
#         path (str): The path to the directory where files are searched.
#         regex (str): The regular expression pattern to match filenames against.

#     Returns:
#         list of str: A list of file paths that match the regex pattern.
#     """
#     pattern = re.compile(regex)
#     matched_files = []
#     for dirpath, _, filenames in os.walk(path):
#         for filename in filenames:
#             if pattern.search(filename):
#                 matched_files.append(os.path.join(dirpath, filename))
#     return matched_files


## Example of Code Produced By Crew

This below code was produced on May 22, 2024.

In [11]:
def fibonacci(n):
    """
    Calculate the 'n'th Fibonacci number.

    Args:
        n (int): The position of the Fibonacci number to calculate.

    Returns:
        int: The 'n'th Fibonacci number.

    Example:
        >>> fibonacci(5)
        3

    Description:
        This function efficiently calculates the 'n'th Fibonacci number using a loop instead of recursion.
        It's a well-structured Python function that generates the sequence of Fibonacci numbers up to the given position.

    Notes:
        * The input 'n' must be a positive integer.
        * For negative inputs or non-integer inputs, the function may not behave as expected.
    """
    sequence_values = [0, 1]
    for _ in range(n - 1):
        sequence_values.append(sequence_values[-1] + sequence_values[-2])
    return sequence_values[-1]


def merge_sequences(array1, array2):
    """Merges two input sequences into one sorted sequence.

    Args:
        array1 (list): The first input sequence.
        array2 (list): The second input sequence.

    Returns:
        list: A merged and sorted sequence of the input arrays.

    Description:
        This function merges two input sequences into one sorted sequence. It does this by comparing elements from both sequences
        and appending the smaller one to the result. If one sequence is exhausted, it appends the remaining elements from the other.
        
        The function assumes that the input sequences are lists of comparable elements (e.g., integers or strings).
    """
    merged_result = []
    index1, index2 = 0, 0
    while index1 < len(array1) and index2 < len(array2):
        if array1[index1] < array2[index2]:
            merged_result.append(array1[index1])
            index1 += 1
        else:
            merged_result.append(array2[index2])
            index2 += 1
    merged_result.extend(array1[index1:])
    merged_result.extend(array2[index2:])
    return merged_result


def g(factorial_input, remainder):
    """Calculates the factorial of a given input number using an iterative approach.
    
    This function repeatedly divides the input number by the remainder until the remainder becomes 0. At each iteration, the original input number and the current remainder are swapped. The final result is the calculated factorial.
    
    Args:
        x (int): The input number for which the factorial needs to be calculated.
        y (int): The initial remainder value.

    Returns:
        int: The calculated factorial of the input number.
    
    Example:
        >>> g(5, 0)
        120
    """
    while remainder != 0:
        x, remainder = remainder, x % remainder
    return x


def maximum_sum_subarray(subarrays):
    """Calculate the maximum sum of subarrays in a given list.

    Args:
        subarrays (list): A list of subarrays, where each subarray is a list of numbers.

    Returns:
        int: The maximum sum of all subarrays.

    Example:
         >>> maximum_sum_subarray([[1, 2, 3], [4, 5, 6], [-1, -2, -3]])
        12

    """
    greatest_sum, current_sum = float('-inf'), 0
    for array in subarrays:
        current_sum = max(array, current_sum + array)
        greatest_sum = max(greatest_sum, current_sum)
    return greatest_sum


def calculate_factorial(n):
    """Calculates the factorial of a given integer.

    Args:
        n (int): The input integer for which to calculate the factorial.

    Returns:
        int: The factorial value of 'n'.

    Examples:
        >>> calculate_factorial(0)
        1
        >>> calculate_factorial(5)
        120

    Notes:
        This function uses a recursive approach to calculate the factorial.
        The base case is when 'n' equals 0, in which case the function returns 1.
        For all other values of 'n', the function calls itself with 'n-1' until it reaches the base case.

    """
    return 1 if n == 0 else n * calculate_factorial(n - 1)


def read_csv_file(file_path):
    """
    Reads a CSV file and returns its contents as a list of dictionaries.
    
    Each dictionary represents a row in the CSV file, with keys corresponding to the column headers and values being the cell data.
    
    Args:
        file_path (str): The path to the CSV file to read.
        
    Returns:
        A list of dictionaries, where each dictionary represents a row in the CSV file.
        
    Example:
        >>> read_csv_file('example.csv')
        [{'column1': 'value1', 'column2': 'value2'}, {'column1': 'value3', 'column2': 'value4'}]
    
    Note: The function assumes that the CSV file has a header row, and that the first column contains unique values.
    """
    with open(file_path, mode='r') as csv_file:
        reader = csv.reader(csv_file)
        headers = next(reader)
        data = [dict(zip(headers, row)) for row in reader]
    return data


def query_database(database_name: str, sql_query: str) -> list:
    """
    Execute a SQL query on a SQLite database and return the results.

    Args:
        database_name (str): The name of the SQLite database to connect to.
        sql_query (str): The SQL query to execute on the database.

    Returns:
        list: A list of tuples containing the results of the query.

    Example usage:

    >>> results = query_database('example.db', 'SELECT * FROM users')
    >>> print(results)  # Print the results of the query

    This function establishes a connection to the specified SQLite
    database, executes the provided SQL query using the cursor,
    and returns the results as a list of tuples. It provides a
    simple and efficient way to interact with SQLite databases
    from Python.
    """
    with sqlite3.connect(database_name) as connection:
        cursor = connection.cursor()
        cursor.execute(sql_query)
        results = cursor.fetchall()
    return results


def send_email(user, password, recipient, message):
    """Sends an email using the provided credentials and recipient information.

    Args:
        user (str): The username to use for authentication.
        password (str): The password to use for authentication.
        recipient (str): The email address of the recipient.
        message (str): The body of the email to be sent.

    Returns:
        None
    """
    smtp_server = smtplib.SMTP('smtp.example.com', 587)
    smtp_server.starttls()
    smtp_server.login(user, password)
    smtp_server.sendmail(user, recipient, message)
    smtp_server.quit()


def retrieve_data(url) -> dict:
    """Retrieve data from a given URL.

    Args:
        url (str): The URL to retrieve data from.

    Returns:
        A dictionary containing the retrieved data if the request is successful,
        an empty dictionary otherwise. The status code of the response can be checked
        using the 'status_code' attribute of the response object.

    Raises:
        None
    """
    import requests

    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return {}


def search_paths(root_path: str, regex_pattern: str) -> list:
    """Searches for files in a directory tree that match a given regular expression.

    Args:
        root_path (str): The path to the root directory to start searching from.
        regex_pattern (str): The regular expression pattern to match against file names.

    Returns:
        list: A list of file paths that match the given regular expression.

    Example usage:
         >>> search_paths('/path/to/directory', 'pattern_to_match')
         ['/path/to/directory/file1.txt', '/path/to/directory/subdir/file2.py']

    """
    compiled_pattern = re.compile(regex_pattern)
    results = []
    for directory, _, files in os.walk(root_path):
        for file in files:
            if compiled_pattern.search(file):
                results.append(os.path.join(directory, file))
    return results